In [62]:
import nltk
import numpy as np
from fuzzywuzzy import fuzz

from collections import defaultdict
from numpy.linalg import norm
from numpy import dot
from scipy import stats

In [63]:
import os

#download tokenizer
current_directory = os.getcwd()
path=current_directory+'/nltk_data'
nltk.download('punkt_tab',download_dir=path)
nltk.download('wordnet',download_dir=path)
nltk.download('omw-1.4',download_dir=path)
nltk.data.path.append(path)

[nltk_data] Downloading package punkt_tab to /Users/prosperplayoust/De
[nltk_data]     sktop/Oulu/NLP/Lab2/NLP/labs/lab2/Live/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/prosperplayoust/Desk
[nltk_data]     top/Oulu/NLP/Lab2/NLP/labs/lab2/Live/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/prosperplayoust/Desk
[nltk_data]     top/Oulu/NLP/Lab2/NLP/labs/lab2/Live/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## 2)

In [64]:
#read files
with open('abstracts.txt','r') as f:
    data=f.read().lower().strip()

punctuation=['"','!','.',',',"'",'(',')',';','``',"''",'?','_',':','-']
punctuation=[]

data=data.split('-next-')
data=[ab.strip().split('\n')[:] for ab in data][:]

#remove punctuation from abstracts
for punct in punctuation:
    for i in range(len(data)):
        data[i][2]=data[i][2].replace(punct,'')

#split authors and keywords
for i in range(len(data)):
    data[i][1]=data[i][1].split(',')
    data[i][3]=data[i][3].split(',')
data

[['advancements and challenges of fuel cell integration in electric vehicles: a comprehensive analysis',
  ['manpreet singh',
   ' manish kumar singla',
   ' murodbek safaraliev',
   ' kulwinder singh',
   ' ismoil odinaev',
   ' amir abdel menaem'],
  'fuel cell technology emerges as a promising green solution, offering mitigation to global warming, air pollution, and energy crises. this eco-friendly approach is witnessing a surge in adoption within the automotive sector, with fuel cell buses, cars, scooters, forklifts, and more, becoming increasingly prevalent. the automobile industry has been rapidly advancing fuel cell technology, inching closer to the commercialization of fuel cell vehicles. as various technical hurdles are surmounted and costs are reduced, fuel cell vehicles are poised to become a competitive force in the automobile market, presenting an excellent solution for environmental sustainability and energy efficiency. this review paper delves into the fundamentals of fu

In [65]:
len(data)

31

In [66]:
#custom dictionary class to conveniently add new items
class inverted_index_dict:
    def __init__(self) -> None:
        self.dict=defaultdict()
        
    def add_item(self,token,doc):
        if self.dict.__contains__(token):
            if doc not in self.dict[token]:
                self.dict[token].append(doc)
        else:
            self.dict[token]=[doc]
    
inverted_index=inverted_index_dict()

for j,ab in enumerate(data):
    for i,kw in enumerate(ab[3]):
        inverted_index.add_item(kw.strip(),j)

inverted_index.dict

defaultdict(None,
            {'fuel cell': [0],
             'hydrogen': [0],
             'electric vehicles': [0],
             'renewable energy': [0],
             'sustainability': [0],
             'chronic kidney disease': [1],
             'acute kidney injury': [1],
             'failed repair': [1],
             'adaptive cell state': [1],
             'maladaptive repair': [1],
             'micro-supercapacitors': [2],
             'laser-induced graphene': [2],
             'fluorinated polyimide': [2],
             'phosphorus doping': [2],
             'artificial intelligence': [3,
              15,
              17,
              18,
              19,
              20,
              21,
              22,
              23,
              24,
              25,
              27,
              28,
              29],
             'chatgpt-4': [3],
             'omnie-learning': [3],
             'gemini 1.5': [3],
             'prolarge language model': [3],
             'm

In [67]:
#ordering
kk=list(inverted_index.dict.items())
kk=sorted(kk,key=lambda x: x[0])
kk

[('accurate identification of bismuth brass alloy', [6]),
 ('acute kidney injury', [1]),
 ('adaptive cell state', [1]),
 ('advantageous featureset', [10]),
 ('ai definition', [26]),
 ('ai detection', [21]),
 ('ai regulation', [26]),
 ('annual variation', [5]),
 ('artificial intelligence',
  [3, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29]),
 ('atmospheric turbulence', [11]),
 ('attitudes towards artificial intelligence', [19]),
 ('automation', [27]),
 ('big data', [29]),
 ('big five', [19]),
 ('bioethics', [18]),
 ('blockchain', [18]),
 ('boosting strategy', [10]),
 ('cancer', [24]),
 ('carbon dioxide concentration', [5]),
 ('chatgpt', [21]),
 ('chatgpt-4', [3]),
 ('chronic kidney disease', [1]),
 ('climate change', [4]),
 ('cognitive style', [15]),
 ('computational thinking', [15]),
 ('condensenet', [14]),
 ('convolutional neural networks(cnn)', [14]),
 ('cross-attention', [30]),
 ('cybersecurity', [18]),
 ('daytime dysfunction', [12]),
 ('deep generative models', [22]),
 ('dee

In [68]:
'''
as suggested by the TA, if we have multiple words as keyword (e.g. artificial intelligence),
we split the keyword as 'artificial' and 'intelligence'
'''
#concatenate keywords
concatenated_keywords=set([word.strip() for i,ab in enumerate(data) for word in ab[3]])
T='artificial intelligence'
output='T: artificial intelligence\n'
#flatten each word in keyword
keywords=[k.split() for k in concatenated_keywords]
keywords=[p for x in keywords for p in x]
keywords=set(keywords)
X=[[k,0] for k in keywords]

for i,kw in enumerate(keywords):
    simil=min(nltk.distance.edit_distance('artificial',kw),nltk.distance.edit_distance('intelligence',kw))
    X[i][1]=simil
    output+=f'{kw}: {simil}\n'
X=sorted(X,key=lambda x: x[0])
print(output)

T: artificial intelligence
democracy: 9
trusted: 9
learning: 9
strategy: 9
injury: 9
convolutional: 10
dispatching: 9
listing: 8
folk: 9
fear: 8
polyimide: 8
hybrid: 9
(dnns): 10
automation: 7
spatio-temporal: 11
neural: 8
cell: 8
hazard: 10
fluorinated: 10
activities: 5
featureset: 10
knowledge: 8
oxidation: 8
kidney: 9
catalysis: 8
cybersecurity: 11
femtosecond: 9
perioperative: 11
metaphorfree: 10
structural: 8
systems: 10
primary: 8
embedding: 9
hydrological: 8
chronic: 8
belief: 8
rules: 9
state: 9
interpretability: 12
failed: 9
condensenet: 9
domain: 9
dioxide: 8
identification: 8
artificial: 0
flood: 9
personalized: 10
bismuth: 9
photocatalysts: 12
convolutions: 10
weather: 10
innovation: 9
extreme: 9
definition: 7
spectroscopy: 10
contribution: 9
life: 8
multi-modality: 10
models: 10
enabling: 8
valley: 9
1.5: 10
nomadic: 8
exposure: 10
interpretation: 10
concentration: 11
model: 9
semi-creative: 11
variation: 7
effective: 8
personality: 9
duoethnography: 12
blockchain: 9
distu

In [69]:
print(X)
len(X),len(keywords)

[['(dnns)', 10], ['1.5', 10], ['accurate', 9], ['activities', 5], ['acute', 8], ['adaptive', 8], ['advantageous', 10], ['ai', 8], ['alloy', 9], ['analysis', 8], ['annual', 7], ['approach', 8], ['artificial', 0], ['atmospheric', 10], ['attitudes', 7], ['automation', 7], ['belief', 8], ['big', 9], ['bioethics', 9], ['bismuth', 9], ['blockchain', 9], ['boosting', 9], ['brass', 9], ['breakdown', 9], ['cancer', 9], ['carbon', 9], ['catalysis', 8], ['cell', 8], ['change', 10], ['chatgpt', 10], ['chatgpt-4', 10], ['chronic', 8], ['climate', 9], ['cognitive', 8], ['communication', 10], ['computational', 9], ['concentration', 11], ['condensenet', 9], ['context', 10], ['contribution', 9], ['convolutional', 10], ['convolutions', 10], ['coupling', 9], ['cross-attention', 12], ['cybersecurity', 11], ['data', 8], ['daytime', 8], ['deep', 10], ['definition', 7], ['democracy', 9], ['densenet', 9], ['dependence', 7], ['detection', 8], ['diabetes', 9], ['different', 9], ['digital', 6], ['dioxide', 8], [

(228, 228)

## 3)

In [70]:
Y=[]
concatenated_keywords=set([word.strip() for i,ab in enumerate(data) for word in ab[3]])

split_kk=[[k[0].split(),k[1]] for k in kk]
for kws,ids in split_kk:
    for kw in kws:
        for id in ids:
            simils=[]
            for word in data[id][0].split():
                simils.append(nltk.distance.edit_distance(word,kw))
            simil=min(simils)
            Y.append([kw,id,simil])
#each element: keyword, document it appears in, edit distance with the title of that document

In [71]:
Y=sorted(Y,key=lambda x: x[0])
i=0
new_Y=[]
w=Y[0][0]
distances=[]
while i<len(Y):
    if Y[i][0]!=w:
        new_Y.append([w,min(distances)])
        w=Y[i][0]
        distances=[Y[i][2]]
    else:
        distances.append(Y[i][2])
    i+=1
new_Y.append([w,min(distances)])
new_Y

[['(dnns)', 5],
 ['1.5', 0],
 ['accurate', 0],
 ['activities', 5],
 ['acute', 4],
 ['adaptive', 6],
 ['advantageous', 8],
 ['ai', 2],
 ['alloy', 4],
 ['analysis', 0],
 ['annual', 0],
 ['approach', 7],
 ['artificial', 0],
 ['atmospheric', 2],
 ['attitudes', 0],
 ['automation', 6],
 ['belief', 1],
 ['big', 3],
 ['bioethics', 2],
 ['bismuth', 0],
 ['blockchain', 6],
 ['boosting', 4],
 ['brass', 0],
 ['breakdown', 0],
 ['cancer', 3],
 ['carbon', 4],
 ['catalysis', 0],
 ['cell', 0],
 ['change', 4],
 ['chatgpt', 4],
 ['chatgpt-4', 0],
 ['chronic', 5],
 ['climate', 6],
 ['cognitive', 8],
 ['communication', 0],
 ['computational', 9],
 ['concentration', 9],
 ['condensenet', 7],
 ['context', 6],
 ['contribution', 7],
 ['convolutional', 9],
 ['convolutions', 9],
 ['coupling', 0],
 ['cross-attention', 9],
 ['cybersecurity', 11],
 ['data', 0],
 ['daytime', 0],
 ['deep', 0],
 ['definition', 5],
 ['democracy', 8],
 ['densenet', 5],
 ['dependence', 7],
 ['detection', 0],
 ['diabetes', 5],
 ['different

## 4)

In [72]:
npX=np.array([x[1] for x in X])
npY=np.array([y[1] for y in new_Y])
stats.pearsonr(npX,npY)

PearsonRResult(statistic=0.20078155996688274, pvalue=0.002317905394913877)

## 5)

In [73]:
#get all unique keywords
concatenated_keywords=set([word.strip() for i,ab in enumerate(data) for word in ab[3]])
keywords=[k.split() for k in concatenated_keywords]
keywords=[p for x in keywords for p in x]
keywords=list(set(keywords))
M=np.zeros((len(data),len(keywords)))

for i,kw in enumerate(keywords):
    for j,ab in enumerate(data):
        abstract=data[j][2]
        num=abstract.count(kw)
        M[j,i]=num

M

array([[ 0.,  0.,  0., ...,  0., 10.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  2.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

## 6)

In [74]:
def boolean_matching(query,db=M,kws=keywords):
    qkeys=query.split()
    result=np.ones(len(data))
    for qkey in qkeys:
        i=kws.index(qkey)
        doc_res=db[:,i]
        doc_res[doc_res>0]=1
        result=np.logical_and(result,doc_res)
    return result

query='artificial intelligence'
boolean_matching(query)

array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True, False])

## 7)

In [75]:
n=len(data)
df=np.zeros(len(keywords))
for i,col in enumerate(M.T):
    df[i]=len(col[col>0])

idf=np.zeros(len(keywords))
for i,f in enumerate(df):
    if df[i]==0:
        idf[i]=0
    else:
        idf[i]=np.log10(n/f)


tfidf_M=M*idf

def tfidf_query_indexer(query,tfidf=tfidf_M,kws=keywords,df=df):
    qkeys=query.split()
    indexed_query=np.zeros(len(keywords))
    
    for qkey in qkeys:
        if qkey not in keywords:
            continue
        i=kws.index(qkey)
        tf=qkeys.count(qkey)
        idf=np.log10(n/df[i])

        indexed_query[i]=tf*idf
    return indexed_query        
        


query='artificial intelligence'
indexed_query=tfidf_query_indexer(query)




In [76]:
similiraty_scores=[]*len(data)

for row in tfidf_M:
    sim_score=dot(indexed_query,row) / (norm(indexed_query)*norm(row))
    similiraty_scores.append(sim_score)

print(similiraty_scores)

print(f"Most similar document : {np.argmax(similiraty_scores)}")


[0.0, 0.0, 0.0, 0.1466687769410364, 0.0, 0.0, 0.0, 0.02781090757631393, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034644578952261446, 0.047189801155514764, 0.10201805788630522, 0.09884296434528193, 0.05423442239741883, 0.015734361749655604, 0.0, 0.057074720189032944, 0.07540067344049044, 0.1978484267038181, 0.08419364250539287, 0.06263564055326303, 0.08916239181941124, 0.03615844322371598, 0.0426269667537797, 0.0]
Most similar document : 24


# 8

In [77]:
Z = []*len(data)

similarity_fw_abs_title = 0

for doc in data:
    title=doc[0]
    abstract=doc[2]
    ratio = fuzz.partial_ratio(title, abstract)
    if ratio > 80:
        similarity_fw_abs_title +=1
    Z.append(ratio)
    
print(Z)

print(f"There are {similarity_fw_abs_title} accepted matches with fuzzywuzzy on {len(data)} documents")


[43, 51, 28, 71, 32, 41, 49, 41, 42, 47, 55, 48, 46, 41, 30, 52, 34, 49, 33, 31, 58, 33, 41, 37, 73, 57, 37, 25, 72, 43, 37]
There are 0 acctepted matches with fuzzywuzzy on 31 documents


We see that all our documents' title don't match precisely with the content of the corresponding abstract. In general, with our data, the words present in title don't often appear in the abstract.
To get more matches, we could lower the threshold to 70%, and we would get 3 matches, but this only depends on the data you select. 